ClearML Template

In [3]:
!pip install clearml

from clearml import Task, Dataset

import pandas as pd
import os
import time


os.environ["CLEARML_API_ACCESS_KEY"] = "BF3OQG84RHSYHO86OIYII6Y7VVVMDV"
os.environ["CLEARML_API_SECRET_KEY"] = "7qQDrf96hXOsbmEio_kUbkEjUqqKaMyq4HB4K1fzOWHjKPelw2HEJdB3RrUvh4FZlA8"
os.environ["CLEARML_API_HOST"] = "https://api.clear.ml"
os.environ["CLEARML_WEB_HOST"] = "https://app.clear.ml"
os.environ["CLEARML_FILES_HOST"] = "https://files.clear.ml"


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [24]:
testDataPath = Dataset.get(dataset_id="a5958fdaf8cd449bbadc2bf57e5e797b").get_local_copy()
print(testDataPath)

/Users/sebastian/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b


In [26]:
test_val_train_folder=os.listdir(testDataPath)

print(testDataPath+"/"+test_val_train_folder[0])
print(testDataPath+"/"+test_val_train_folder[1])
print(testDataPath+"/"+test_val_train_folder[2])

for file in test_val_train_folder:
    if "train" in file:
        train_file = file
    elif "valid" in file:
        valid_file = file
    elif "test" in file:
        test_file = file

/Users/sebastian/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/train_filtered.tsv
/Users/sebastian/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/valid_filtered.tsv
/Users/sebastian/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/test_filtered.tsv


In [27]:
# wczytujemy do DataFrame
columns = [
    "id", "label", "statement", "subjects", "speaker", "speaker_job", "state_info",
    "party_affiliation", "barely_true", "false", "half_true",
    "mostly_true", "pants_on_fire", "context"
]

df_train = pd.read_csv(f"{testDataPath}/{train_file}", sep='\t', names=columns, header=None)
df_valid = pd.read_csv(f"{testDataPath}/{valid_file}", sep='\t', names=columns, header=None)
df_test = pd.read_csv(f"{testDataPath}/{test_file}", sep='\t', names=columns, header=None)

print("train:", df_train.shape)
print("valid:", df_valid.shape)
print("test:", df_test.shape)


train: (3681, 14)
valid: (432, 14)
test: (461, 14)


In [28]:
df_test.head(1)

,id,label,statement,subjects,speaker,speaker_job,state_info,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,11972.json,True,Building a wall on the U.S.-Mexico border will...,immigration,rick-perry,Governor,Texas,republican,30,30,42,23,18,Radio interview


# Pre processing


### Importy

In [50]:
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import spacy

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Stop words

In [51]:
stop_words = set(stopwords.words('english'))

In [52]:
#2 wersja z użyciem spacy

# nlp = spacy.load('en_core_web_sm')

#można tez połączyć te dwie metody robiąc listę każdej i potem .concat()

### Czyszczenie tekstu

In [53]:
def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'\d+', '', text)  # remove all digits from text
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'<[^>]+>', '', text)  # remove html tags
    text = re.sub(r'<[^>]*>', '', text)  # additional html tag removal
    emojis = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)  # find emojis
    text = re.sub(r'[\W]+', ' ', text) + ' '.join(emojis).replace('-', '')
    text = text.strip() # remove
    return text

### Lematyzacja

In [54]:
lemmatizer = WordNetLemmatizer()

### Stemming

In [55]:
porter = PorterStemmer()

### Tokenizacja + poprzednie funkcje

In [56]:
def custom_tokenizer(text, cfg):
    text = clean_text(text)
    tokens = word_tokenize(text)

    if cfg["lemmatization"]:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    if cfg["stemming"]:
        tokens = [porter.stem(token) for token in tokens]
    if cfg["remove_stopwords"]:
        tokens = [t for t in tokens if t not in stop_words]
    tokens = [t for t in tokens if len(t) >= cfg["min_token_length"]]

    if cfg["concat_to_sentence"]:
        return " ".join(tokens)
    return tokens


# Metoda Embedding

# Metoda Klasyfikacji

# Wywołanie Taska

In [57]:
task_name = "Wpisać nazwe Taska"

# Tworzenie taska
task = Task.init(project_name="FakeNewsDetection", task_name=task_name)

# config do preprocesingu
cfg_preprocessing ={
    "tokenizer": True,
    "lemmatization": True,
    "stemming": False,
    "remove_stopwords": True,
    "min_token_length": 3, # wywala slowa któtsze niz 3
    "concat_to_sentence": False # czy łaczymy w zdania (True) czy zostawimy jako wyrazy (False)
}

# preprocessing
df_train["tokens"] = df_train["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))
df_valid["tokens"] = df_valid["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))
df_test["tokens"] = df_test["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))


# wyswietl przyklad preprocesingu
print(df_train[["statement", "tokens"]].head(3))


# metoda
start_time = time.time()

# Tutaj kod ..... = history = model.fit(...)

end_time = time.time()
training_duration = end_time - start_time

# Tutaj kod ..... = test_loss, test_accuracy = model.evaluate(X_test, y_test)

logger = task.get_logger()
for epoch in range(len(history.history['loss'])):
    logger.report_scalar("Loss", "train", value=history.history['loss'][epoch], iteration=epoch)
    logger.report_scalar("Loss", "val", value=history.history['val_loss'][epoch], iteration=epoch)
    logger.report_scalar("Accuracy", "train", value=history.history['accuracy'][epoch], iteration=epoch)
    logger.report_scalar("Accuracy", "val", value=history.history['val_accuracy'][epoch], iteration=epoch)

logger.report_scalar("Evaluation", "Test Loss", value=test_loss, iteration=0)
logger.report_scalar("Evaluation", "Test Accuracy", value=test_accuracy, iteration=0)

logger.report_scalar("Training", "Duration (s)", value=training_duration, iteration=0)

# Zamykanie taska
task.close()

ClearML Task: created new task id=008e4e0c44414f4798f27553ab849c3a
ClearML results page: https://app.clear.ml/projects/25cc2e9801f0421ba3bf3ef6bcb791c7/experiments/008e4e0c44414f4798f27553ab849c3a/output/log
ClearML results page: https://app.clear.ml/projects/25cc2e9801f0421ba3bf3ef6bcb791c7/experiments/008e4e0c44414f4798f27553ab849c3a/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
                                                                                                                                                     statement  \
0                                                                           Says the Annies List political group supports third-trimester abortions on demand.   
1                                                                               Health care reform legislation is likely to mandate free sex change surgeries.   
2  The Chicago Bears have had more starting quarterbacks in the last 10 year

NameError: name 'history' is not defined